In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langgraph.graph import StateGraph, END, START
from typing import TypedDict, Literal
from pydantic import BaseModel, Field

In [3]:
load_dotenv()
llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash")

In [5]:
class SentimentSchema(BaseModel):
    sentiment: Literal["positive","negative"] = Field(description="sentiment of the review")

In [13]:
class DiagnosisSchema(BaseModel):
    issue_type: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(description='The category of issue mentioned in the review')
    tone: Literal["angry", "frustrated", "disappointed", "calm"] = Field(description='The emotional tone expressed by the user')
    urgency: Literal["low", "medium", "high"] = Field(description='How urgent or critical the issue appears to be')

In [7]:
struc_llm = llm.with_structured_output(SentimentSchema)

In [11]:
struc_llm.invoke("hii").sentiment

'positive'

In [9]:
class ReviewState(TypedDict):
    review: str
    sentiment: Literal["positive","negative"]
    response: str
    diagnosis: str

In [ ]:
def find_sentiment(state:ReviewState):
    prompt = f"Find the sentiment of the following review: \n {state["review"]}"
    output = struc_llm.invoke(prompt).sentiment

    return {"sentiment":output}

def cheackpoint(state: ReviewState)-> Literal["positive_response","run_diagnosis"]:
    if state["review"] == "positive":
        return "positive_response"
    else:
        return "run_diagnosis"
    
def positive_response(state: ReviewState):

    prompt = f'''Write a warm thank-you to the response of the review: 
    {state["review"]}.
    and kindly ask for feedback on our website'''
    
    output = llm.invoke(prompt).content
    return {"response":output}

def run_diagnosis(state: ReviewState):
    prompt = 